In [1]:
import pandas as pd

root = '../dataset/hi'

# Load dataset
hiv = pd.read_csv(f'{root}/hiv-charge-CS2_36.csv')
hii = pd.read_csv(f'{root}/hii-charge-CS2_36.csv')

In [2]:
# Unisci i due dataset utilizzando la colonna "cycle"
df = pd.merge(hiv, hii, on="cycle")

# Rimuovi la colonna "cycle" dal DataFrame risultante
df = df.drop(columns=["cycle"], inplace=False)

# Visualizza il DataFrame risultante
print(df)

         hi_v      hi_i
0    7.091993  0.125641
1    6.639214  0.121732
2    6.599330  0.120788
3    6.627282  0.120991
4    6.389022  0.121652
..        ...       ...
955  0.371274  0.109306
956  0.345076  0.109547
957  0.392511  0.110228
958  0.404034  0.110994
959  0.399089  0.110416

[960 rows x 2 columns]


In [3]:
# Split into train and test sets
train_size = int(len(df) * 0.8)
train_data, test_data = df[:train_size], df[train_size:]

In [4]:
from sklearn.preprocessing import StandardScaler

# Istanzia lo StandardScaler
scaler = StandardScaler()

# Standardizza i dati
scaled_data = scaler.fit_transform(df)

# Crea un nuovo DataFrame con i dati standardizzati
df_standardized = pd.DataFrame(scaled_data, columns=df.columns)


c:\Python39\lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\Python39\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Python39\lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Python39\lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.appl

In [5]:
df

,hi_v,hi_i
0,7.091993,0.125641
1,6.639214,0.121732
2,6.599330,0.120788
3,6.627282,0.120991
4,6.389022,0.121652
...,...,...
955,0.371274,0.109306
956,0.345076,0.109547
957,0.392511,0.110228
958,0.404034,0.110994


In [37]:
import numpy as np

def create_sequences(data, seq_length):
    X = []
    y = []
    for i in range(len(data) - seq_length):
        seq = data[['hi_v', 'hi_i']].iloc[i:i+seq_length].values
        target = data[['hi_v','hi_i']].iloc[i+seq_length]
        X.append(seq)
        y.append(target)
    return np.array(X), np.array(y)

In [38]:
seq_length = 10

# Create sequences for training set
X_train, y_train = create_sequences(train_data, seq_length)

# Create sequences for testing set
X_test, y_test = create_sequences(test_data, seq_length)

# Reshape input data
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 2))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 2))

In [39]:

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense


In [40]:

model = Sequential()
model.add(LSTM(units=128, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))

model.add(Dense(units=1))


In [41]:

model.compile(loss='mean_squared_error', optimizer='adam')


In [42]:

model.fit(X_train, y_train, epochs=100, batch_size=1, verbose=2)


Epoch 1/100


ValueError: in user code:

    File "c:\Python39\lib\site-packages\keras\engine\training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "c:\Python39\lib\site-packages\keras\engine\training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Python39\lib\site-packages\keras\engine\training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "c:\Python39\lib\site-packages\keras\engine\training.py", line 890, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "c:\Python39\lib\site-packages\keras\engine\training.py", line 948, in compute_loss
        return self.compiled_loss(
    File "c:\Python39\lib\site-packages\keras\engine\compile_utils.py", line 201, in __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    File "c:\Python39\lib\site-packages\keras\losses.py", line 139, in __call__
        lo

In [ ]:

plt.plot(history.history['loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.show()
